#### Importing Computer Vision Libraries

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

import time


In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

#### Preparing the data

In [ ]:
#training data

train_data = datasets.FashionMNIST(root='data', train= True, transform= ToTensor(), download= True)
test_data = datasets.FashionMNIST(root='data', train= False, transform= ToTensor(), download= True)

In [ ]:
print(f" the length of training data is {len(train_data)}, length of test data  { len(test_data)}")

In [ ]:
#visualize any image randomly
img, label = train_data[2]
img, label

In [ ]:
classnames = train_data.classes
classnames

In [ ]:
#visualize the image

plt.imshow(img.squeeze(), cmap="gray")
plt.title(classnames[label])
plt.axis(False)



In [ ]:
#multiple random images
torch.manual_seed(42)
fig  = plt.Figure(figsize=(9,9))
rows, cols = 4, 4

for i in range(1, rows*cols +1):
    #pick image randomly
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap = 'gray')
    plt.axis(False)
    plt.title(classnames[label])

In [ ]:
#dataloader to loop through the dataset

#hyperparameter
BATCH_SIZE = 32

train_data_batch = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle= True )

test_data_batch = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle= False)

print(f" The reduced training databatch is {len(train_data_batch)}")
print(f" The reduced test databatch is {len(test_data_batch)}")

In [ ]:
# Check out what's inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_data_batch))
train_features_batch.shape, train_labels_batch.shape

#### Creating a baseline model

In [ ]:
class FashionMNISTVO(nn.Module):
    def __init__(self, input_shape: int, hidden_layers: int,  output_shape: int):
        super().__init__()

        self.linear_stack = nn.Sequential( nn.Flatten(),
                                          nn.Linear(in_features= input_shape, out_features= hidden_layers),
                                          nn.Linear(in_features= hidden_layers, out_features= output_shape),
                                          )
        

    #define the forward method
    def forward(self, x):
        return self.linear_stack(x)

In [ ]:
#create a model for the class

model_0 = FashionMNISTVO(input_shape= 784,
                         hidden_layers= 10,
                         output_shape=len(classnames)).to('cpu')

model_0

#### Setup loss, optimizer and evaluation metrics

In [ ]:
#loss function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr = 0.1)

In [ ]:
#import helper functions

import requests
from pathlib import Path

r = requests.get(url='https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')

if Path("helper_functions.py").is_file():
    print(" Skipping the download the file already exists")
else:
    print('Downloading file...........')

    with open('helper_functions.py', 'wb') as f:

        f.write(r.content)

In [ ]:
#time function

def my_run_time(start_time, end_time):

    return f" the elapsed time is { end_time - start_time} seconds"
    

In [ ]:
from tqdm.auto import tqdm

#set the manual seed
torch.manual_seed(42)
#set the timer
start_time = time.time()

#set the epochs
epochs = 3

for epoch in tqdm(range(epochs)):

    print(f" this is epoch number {epoch}")

    train_loss = 0

    for batch, (X,y) in enumerate(train_data_batch):

        #train model
        model_0.train()

        #do the forward pass
        y_train_pred = model_0(X)

        #calculate the loss
        loss = loss_fn(y_train_pred, y)
        train_loss += loss

        #zero gradient

        optimizer.zero_grad()

        #loss backward
        loss.backward()

        #optimze step
        optimizer.step()

        if batch % 500 == 0:
            print(f"the batches checked are {batch * len(X)}")

        train_loss /= len(train_data_batch)

        #testing

        test_loss, acc = 0, 0

    model_0.eval()

    with torch.inference_mode():

            #do forward pass

        for batch, (X,y) in enumerate(test_data_batch):
                #do the forward pass
            y_test = model_0(X)

                #calculate the loss
            t_loss = loss_fn(y_test, y)
            test_loss += t_loss

        test_loss /= len(test_data_batch)

    print(f" the training loss is {train_loss:.6f}        <||||     test loss {test_loss:.6f}")

end_time = time.time()


print(my_run_time(start_time, end_time))

#### Adding Non-Linearity to our Model


In [ ]:
class FashionMNISTV1(nn.Module):

    def __init__(self, input_shape: int, hidden_layers : int, output_shape: int ):
        super().__init__()

        self.linear_stacktwo = nn.Sequential( nn.Flatten(),
                                             nn.Linear(in_features= input_shape, out_features= hidden_layers),
                                             nn.ReLU(),
                                             nn.Linear(in_features=input_shape, out_features= output_shape),
                                             nn.ReLU())
        

    def forward(self, x: torch.Tensor):
        return self.linear_stacktwo(x)
        

In [ ]:
#instantiate the class

model_1 = FashionMNISTV1(input_shape=784,
                         hidden_layers= 10,
                         output_shape=len(classnames)).to(device='cpu')

#### Loss function and Accuracy metrics

In [65]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model_1.parameters(), lr = 0.1)



#### functionalizing the testing and training loop

In [69]:
def training_step(model: torch.nn.Module,
                  optimizer: torch.optim.Optimizer,
                  loss_fn: torch.nn.Module,
                  dataloader: torch.utils.data.DataLoader,
                  device: torch.device = device):
    

    model.to(device)

    train_acc, train_loss = 0, 0

    for batch, (X,y) in enumerate(dataloader):

        #do the forward pass

        y_pred = model(X)

        #calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss

        #optimizer

        optimizer.zero_grad()

        #loss backward
        loss.backward()

        #optimizer step
        optimizer.step()

        print(f"the train loss is {train_loss:.5f}     || ")

    
    

In [ ]:
def test_step(model: torch.nn.Module,
                  optimizer: torch.optim.Optimizer,
                  loss_fn: torch.nn.Module,
                  dataloader: torch.utils.data.DataLoader,
                  device: torch.device = device):
    model.to(device)

    model.eval()


    #loop through the dataset data

    with torch.inference_mode():

        for X, y in dataloader:

            #forwward pass

            y_test = model(X)

            #calculate the loss

            loss = loss_fn(y_test, y)

        print(f"the train loss is {train_loss:.5f}     || ")

           


